# Data generation for ANN recognition with ANN masks

This notebook generates unmasked and masked data for the ANN recognition with ANN masks experiments.

In [1]:
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy import ndimage
import matplotlib.pyplot as plt
import fnmatch
import os
import itertools
import cv2
import pathlib

In [2]:
alexnet_sggbp_fls = list(pathlib.Path('../out/new_maps_imagenet').glob('**/*network_alexnet_method_smoothgradguidedbp.mat'))
alexnet_gradcam11_fls = list(pathlib.Path('../out/new_maps_imagenet').glob('**/*network_alexnet_method_gradcam_selector_11.mat'))
image_fls = list(pathlib.Path('../human-maps').glob('**/*image.mat'))
ids = list(map(lambda f: int(str(f).split('/')[-1].split('_')[0][3:]), image_fls))
image_fls[:2], alexnet_sggbp_fls[:2], alexnet_gradcam11_fls[:2]

([PosixPath('../human-maps/img24_image.mat'),
  PosixPath('../human-maps/img126_image.mat')],
 [PosixPath('../out/new_maps_imagenet/img146/img146_network_alexnet_method_smoothgradguidedbp.mat'),
  PosixPath('../out/new_maps_imagenet/img115/img115_network_alexnet_method_smoothgradguidedbp.mat')],
 [PosixPath('../out/new_maps_imagenet/img146/img146_network_alexnet_method_gradcam_selector_11.mat'),
  PosixPath('../out/new_maps_imagenet/img115/img115_network_alexnet_method_gradcam_selector_11.mat')])

Helper functions for loading, saving, and mask/image processing.

In [3]:
which_files = {
    'alexnet-sggbp': alexnet_sggbp_fls,
    'alexnet-gradcam11': alexnet_gradcam11_fls,
}

def get_mask(mat_fname, exponent, threshold, rotation, mp_shape):
    """Load the mask and apply transformations."""

    # Path to specific (patch) map.
    mat_contents = sio.loadmat(mat_fname)
    #TODO: remove hardcode
    try:
        mp = mat_contents['smoothgradguidedbp']
    except KeyError:
        mp = mat_contents['gradcam']
    
    # Resize.
    mp = cv2.resize(mp, mp_shape, interpolation=cv2.INTER_AREA)
    
    # Truncate.
    if threshold is None:
        # Truncate 50% of pixels.
        half = np.prod(mp.shape) // 2
        threshold = np.partition(mp, np.prod(mp.shape)//2, axis=None)[half]
    mp[mp < threshold] = threshold
    mp -= threshold

    # Normalize.
    mp /= np.max(mp)

    # Exponentiate.
    mp **= exponent
    
    # Rotate.
    mp = np.rot90(mp, rotation // 90)
    
    return mp
 
    
def get_image(im_fname, im_shape):
    """Load the image, resize, and tranform to float."""
    
    # Read in image.
    im_contents = sio.loadmat(im_fname)
    im = im_contents['im']

    # Resize.
    im = cv2.resize(im, im_shape, interpolation=cv2.INTER_AREA)

    # Color conversion.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(float)/255 
    
    return im
        

def create_test_data(mask_type, null_type, exponent, threshold, rotation, im_shape=[250, 250]):

    # One folder per hparam setting.
    set_name = 'mask={}_null={}_exp={}_threshold={}_rotation={}'.format(
        mask_type, null_type, exponent, threshold, rotation)
    dir_name = 'ANN_recognition_data'
    set_fn = pjoin(dir_name, set_name)
         
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    if not os.path.exists(set_fn):
        os.mkdir(set_fn)
    
    # Combine the image with each mask (correct and incorrect).
    for image_fl in image_fls:
        
        im = get_image(image_fl, im_shape=im_shape)
        img_id = int(str(image_fl).split('/')[-1].split('_')[0][3:])
                    
        if null_type == 'black':
            null_value = np.array([0.0, 0.0, 0.0])
        elif null_type == 'average':
            null_value = im.mean(axis=0).mean(axis=0)
        else:
            raise ValueError("Unsupported null value type.")
        
        # Loop over possible_masks.
        for mask_id in ids:
            mask_fl = list(filter(lambda f: str(mask_id) in str(f), which_files[mask_type]))[0]
            mp = get_mask(mask_fl, exponent, threshold, rotation, mp_shape=im_shape)

            # Interpolate to null value.
            transformed_im = (
                np.einsum('ij,k->ijk', 1 - mp, null_value) +
                np.einsum('ij,ijk->ijk', mp, im - null_value[np.newaxis, np.newaxis, :])
            )

            # Write image to File
            fn = 'img{}_mask{}.png'.format(img_id, mask_id)        
            file_path = pjoin(set_fn, fn)
            cv2.imwrite(file_path, transformed_im * 255)
        
    # Also create an appropriately-resized unmasked image.
    # This assumes all masks of a certain type are the same size.
    for image_fl in image_fls:
        im = get_image(image_fl, im_shape=mp.shape)
        imid = str(image_fl).split('/')[-1].split('_')[0][3:]
        fn = 'img{}_nomask.png'.format(imid)   
        file_path = pjoin(set_fn, fn)
        cv2.imwrite(file_path, im * 255)

Create the unmasked and masked data. 

In [4]:
null_types = ['black']#, 'average']
mask_types = which_files.keys()
exponents = [1.0]#, 1.5, 2.0, 4.0, 8.0, 16.0]
thresholds = [None] #[0.0625, 0.125, 0.25, 0.5]
rotations = [0]#, 90, 180, 270]

for (mask_type, null_type, exponent, threshold, rotation) in itertools.product(
    mask_types, null_types, exponents, thresholds, rotations):
    print('Doing...')
    create_test_data(mask_type=mask_type, 
                     null_type=null_type,
                     exponent=exponent,
                     threshold=threshold,
                     rotation=rotation)
    print('Done.')

Doing...
Done.
Doing...
Done.
